<a href="https://colab.research.google.com/github/Kh-Harakeh/Thomas-vs-Gauss-Jordan/blob/main/thomas_vs_gauss_jordan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

def validate_dimensions(a, b, c, d, n):
    """
    Validates the dimensions of the input arrays for the tridiagonal system.

    Parameters:
        a (array-like): Sub-diagonal (length n-1)
        b (array-like): Main diagonal (length n)
        c (array-like): Super-diagonal (length n-1)
        d (array-like): Right-hand side (length n)
        n (int): Expected length of the main diagonal.

    Raises:
        ValueError: If dimensions of input arrays are inconsistent.
    """
    if len(b) != n or len(a) != n - 1 or len(c) != n - 1:
        raise ValueError(f"Dimension mismatch: len(b)={len(b)}, len(a)={len(a)}, len(c)={len(c)}, len(d)={n}")

def thomas_algorithm(a, b, c, d, precision_threshold=1e-12):
    """
    Solves a tridiagonal system using the Thomas Algorithm.

    Parameters:
        a (array-like): Sub-diagonal (length n-1)
        b (array-like): Main diagonal (length n)
        c (array-like): Super-diagonal (length n-1)
        d (array-like): Right-hand side (length n)
        precision_threshold (float): Threshold for numerical precision checks.

    Returns:
        x (np.ndarray): Solution vector (length n)

    Raises:
        ValueError: If dimensions of input arrays are inconsistent.
        ValueError: If the matrix is singular or nearly singular.
    """
    n = len(d)

    # Ensure the diagonals have compatible dimensions
    validate_dimensions(a, b, c, d, n)

    # Initialize arrays for forward elimination
    c_prime = np.zeros(n, dtype=float)
    d_prime = np.zeros(n, dtype=float)

    # Forward elimination
    c_prime[0] = c[0] / b[0]
    d_prime[0] = d[0] / b[0]

    for i in range(1, n):
        denominator = b[i] - a[i - 1] * c_prime[i - 1]

        # Stability check with threshold for nearly singular matrix
        threshold = precision_threshold * max(abs(b[i]), abs(a[i - 1] * c_prime[i - 1]), 1)
        if abs(denominator) < threshold:
            raise ValueError("Matrix is singular or nearly singular.")

        c_prime[i] = c[i] / denominator if i < n - 1 else 0
        d_prime[i] = (d[i] - a[i - 1] * d_prime[i - 1]) / denominator

    # Back substitution
    x = np.zeros(n, dtype=float)
    x[-1] = d_prime[-1]

    for i in range(n - 2, -1, -1):
        x[i] = d_prime[i] - c_prime[i] * x[i + 1]

    return x

def generate_tridiagonal_system(n, a_range=(1, 10), b_range=(10, 20), c_range=(1, 10), d_range=(1, 100)):
    """
    Generates a random tridiagonal system of equations.

    Parameters:
        n (int): Dimension of the system (n x n matrix).

    Returns:
        a (np.ndarray): Sub-diagonal (length n-1).
        b (np.ndarray): Main diagonal (length n).
        c (np.ndarray): Super-diagonal (length n-1).
        d (np.ndarray): Right-hand side (length n).
    """
    a = np.random.uniform(a_range[0], a_range[1], size=n-1)  # Sub-diagonal
    b = np.random.uniform(b_range[0], b_range[1], size=n)   # Main diagonal (ensuring dominance)
    c = np.random.uniform(c_range[0], c_range[1], size=n-1)  # Super-diagonal
    d = np.random.uniform(d_range[0], d_range[1], size=n)   # Right-hand side
    return a, b, c, d

# Example usage for a high-dimensional tridiagonal system
if __name__ == "__main__":
    dimension = 1000  # Define the size of the matrix (e.g., 1000 for 1000x1000)
    a, b, c, d = generate_tridiagonal_system(dimension)

    # Solve the system
    try:
        solution = thomas_algorithm(a, b, c, d)
        print("Solution for dimension {}: Success".format(dimension))
        print("First few elements of the solution:", solution[:min(10, len(solution))])
    except ValueError as e:
        print("Error:", e)

Solution for dimension 1000: Success
First few elements of the solution: [  9.86326982  -3.86133245   2.25721665   2.51355565  -1.78668179
  24.58905553 -18.40385401  12.37206678   1.14320314  -4.64762502]


In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import splu

def gauss_jordan_sparse(A, b):
 """
 Solves a linear system using an optimized Gauss-Jordan
 Method with sparse matrix techniques.
 Inputs:
    A: Coefficient matrix (n x n)
    b: Right-hand side vector (length n)
 Outputs:
    x: Solution vector (length n)
 """
 try: # Corrected indentation for try block
  if not isinstance(A, csc_matrix):
   A_sparse = csc_matrix(A)
  else:
   A_sparse = A
   # Perform LU decomposition using sparse techniques
   lu = splu(A_sparse)
   # Solve the system
   x = lu.solve(b)
  return x # Corrected indentation for return statement
 except Exception as e:
  raise ValueError(f"Error during LU decomposition or solving: {e}")